In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import callback_context
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from mongoCRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Austin Animal Center')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
        style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'},
        children=[
            html.Img(src="logo.png", style={'height': '50px', 'marginRight': '10px'}),
            html.H1('Derek Spaulding - SNHU Dashboard', style={'fontWeight': 'bold'})
        ]
    ),
    html.Hr(),
    html.Div(className='filters',
            style={'display' : 'flex'},
             children=[
                 html.Button(id='water-id', n_clicks=0, children='Water Rescue'),
                 html.Button(id='mountain-id', n_clicks=0, children='Mountain Rescue'),
                 html.Button(id='disaster-id', n_clicks=0, children='Disaster Rescue'),
                 html.Button(id='reset-id', n_clicks=0, children='Reset')
             ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_deletable=False,
        row_selectable='single',
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    html.Div([
        html.Div(
            id='map-id',
            style={'flex': '1', 'minWidth': '300px'},
        ),
        html.Div(
            dcc.Graph(id='breed-chart'),
            style={'flex': '1', 'minWidth': '300px'},
        )
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px'})
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
    )
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#Populate the pie chart callback
@app.callback(
    Output('breed-chart', 'figure'),
    Input('datatable-id', 'data')
)
def populate_chart(data):
    if data is None:
        return dash.no_update
    
    dff = pd.DataFrame(data)
    
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    
    total_dogs = breed_counts['count'].sum()
    breed_counts['percentage'] = breed_counts['count'] / total_dogs * 100
    
    breed_counts = breed_counts.sort_values('percentage', ascending=False)

    fig = px.pie(breed_counts, 
                 values='percentage', 
                 names='breed', 
                 title='Dog Breed Composition',
                 hover_data=['count'])
    
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

    return fig

#Filter Buttons
@app.callback(
    Output('datatable-id', 'data'),
    Output('water-id', 'n_clicks'),
    Output('mountain-id', 'n_clicks'),
    Output('disaster-id', 'n_clicks'),
    Output('reset-id', 'n_clicks'),
    [Input('water-id', 'n_clicks'),
    Input('mountain-id', 'n_clicks'),
    Input('disaster-id', 'n_clicks'),
    Input('reset-id', 'n_clicks')],
    prevent_initial_call=True)
def on_click(water_clicked, mountain_clicked, disaster_clicked, reset_clicked):
    df = pd.DataFrame.from_records(shelter.read({}))
        
    if water_clicked == 1:
        df = pd.DataFrame.from_records(shelter.read({
            "$and":[{"breed":{"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                  { "sex_upon_outcome": "Intact Female" },
                  { "age_upon_outcome_in_weeks": {
                      "$gte": 26,
                      "$lte": 156
                  }}]
        }))
    elif mountain_clicked == 1:
        df = pd.DataFrame.from_records(shelter.read({
            "$and":[{"breed":{"$in": ["German Shepherd", "Alaskan Malamute", 
                                      "Old English Sheepdog", "Siberian Husky","Rottweiler"]}},
                  { "sex_upon_outcome": "Intact Male" },
                  { "age_upon_outcome_in_weeks": {
                      "$gte": 26,
                      "$lte": 156
                  }}]
        }))
    
    elif disaster_clicked == 1:
        df = pd.DataFrame.from_records(shelter.read({
            "$and":[{"breed":{"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                                      "Bloodhound", "Rottweiler"]}},
                  { "sex_upon_outcome": "Intact Male" },
                  { "age_upon_outcome_in_weeks": {
                      "$gte": 20,
                      "$lte": 300
                  }}]
        }))
        
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records'), 0, 0, 0, 0


#Update map callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index = 0):
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if dff.size == 0:
        return
    if index is None:
        row = 0
    else: 
        row = index[0]

#Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75,-97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row,4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row,9])
                                 ])
                             ])
               ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:31657/
